<a href="https://colab.research.google.com/github/Haeun-Y/stress-detection/blob/main/Stress_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#stress.csv 불러오기

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("stress.csv")
print(data.head())

          subreddit post_id sentence_range  \
0              ptsd  8601tu       (15, 20)   
1        assistance  8lbrx9         (0, 5)   
2              ptsd  9ch1zh       (15, 20)   
3     relationships  7rorpp        [5, 10]   
4  survivorsofabuse  9p2gbc         [0, 5]   

                                                text     id  label  \
0  He said he had not felt that way before, sugge...  33181      1   
1  Hey there r/assistance, Not sure if this is th...   2606      0   
2  My mom then hit me with the newspaper and it s...  38816      1   
3  until i met my new boyfriend, he is amazing, h...    239      1   
4  October is Domestic Violence Awareness Month a...   1421      1   

   confidence  social_timestamp  social_karma  syntax_ari  ...  \
0         0.8        1521614353             5    1.806818  ...   
1         1.0        1527009817             4    9.429737  ...   
2         0.8        1535935605             2    7.769821  ...   
3         0.6        1516429555       

#No Stress=0, stress=1로 라벨링


In [ ]:
data["label"] = data["label"].map({0: "No Stress", 1: "Stress"})
data = data[["text", "label"]]
print(data.head())

                                                text      label
0  He said he had not felt that way before, sugge...     Stress
1  Hey there r/assistance, Not sure if this is th...  No Stress
2  My mom then hit me with the newspaper and it s...     Stress
3  until i met my new boyfriend, he is amazing, h...     Stress
4  October is Domestic Violence Awareness Month a...     Stress


In [ ]:
print(data.isnull().sum())

text     0
label    0
dtype: int64


#Combination of multiple models and vectorizer

###count+LR

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split



x = np.array(data["text"])
y = np.array(data["label"])

cv = CountVectorizer()
X = cv.fit_transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, 
                                                test_size=0.33, 
                                                random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


LR = LogisticRegression()
LR.fit(xtrain, ytrain)
y_pred=LR.predict(xtest)
print('test accuracy:', accuracy_score(ytest,y_pred))

test accuracy: 0.743863393810032


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


###Count+RandomForest

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split



x = np.array(data["text"])
y = np.array(data["label"])

cv = CountVectorizer()
X = cv.fit_transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, 
                                                test_size=0.33, 
                                                random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

max_accuracy = 0


for x in range(100):
    rf = RandomForestClassifier(random_state=x)
    rf.fit(xtrain,ytrain)
    y_pred = rf.predict(xtest)
    current_accuracy = round(accuracy_score(y_pred,ytest)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x
        
#print(max_accuracy)
#print(best_x)

rf = RandomForestClassifier(random_state=best_x)
rf.fit(xtrain,ytrain)
y_pred = rf.predict(xtest)
print('test accuracy:', accuracy_score(ytest,y_pred))

test accuracy: 0.7331910352187834


###Count+BernoulliNB

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split



x = np.array(data["text"])
y = np.array(data["label"])

cv = CountVectorizer()
X = cv.fit_transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, 
                                                test_size=0.33, 
                                                random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB


model1 = BernoulliNB()
model1.fit(xtrain, ytrain)
y_pred=model1.predict(xtest)
print('test accuracy:', accuracy_score(ytest,y_pred))

test accuracy: 0.7577374599786553


###TFIDF+ Logistic Regression/BernoulliNB/Random Forest

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

x = np.array(data["text"])
y = np.array(data["label"])

tf=TfidfVectorizer()
X=tf.fit_transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, 
                                                test_size=0.33, 
                                                random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


LR = LogisticRegression()
LR.fit(xtrain, ytrain)
y_pred=LR.predict(xtest)
print('test accuracy:', accuracy_score(ytest,y_pred))

test accuracy: 0.7470651013874067


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB

model1 = BernoulliNB()
model1.fit(xtrain, ytrain)
y_pred=model1.predict(xtest)
print('test accuracy:', accuracy_score(ytest,y_pred))

test accuracy: 0.7577374599786553


In [ ]:
from sklearn.ensemble import RandomForestClassifier

max_accuracy = 0


for x in range(100):
    rf = RandomForestClassifier(random_state=x)
    rf.fit(xtrain,ytrain)
    y_pred = rf.predict(xtest)
    current_accuracy = round(accuracy_score(y_pred,ytest)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x
        
#print(max_accuracy)
#print(best_x)

rf = RandomForestClassifier(random_state=best_x)
rf.fit(xtrain,ytrain)
y_pred = rf.predict(xtest)
print('test accuracy:', accuracy_score(ytest,y_pred))

test accuracy: 0.7427961579509071


#bernoulliNB parameter

In [ ]:
from sklearn.naive_bayes import BernoulliNB
model=BernoulliNB()
model.get_params().keys()

dict_keys(['alpha', 'binarize', 'class_prior', 'fit_prior'])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier

model = BernoulliNB(alpha=1)
model.fit(xtrain, ytrain)
y_pred=model.predict(xtest)
print('test accuracy:', accuracy_score(ytest,y_pred))

test accuracy: 0.7577374599786553


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier

model1 = BernoulliNB(fit_prior=True)
model1.fit(xtrain, ytrain)
y_pred=model1.predict(xtest)
print('test accuracy:', accuracy_score(ytest,y_pred))

test accuracy: 0.7577374599786553


In [ ]:
# For training set
from sklearn.metrics import classification_report

print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

   No Stress       0.81      0.64      0.71       444
      Stress       0.73      0.86      0.79       493

    accuracy                           0.76       937
   macro avg       0.77      0.75      0.75       937
weighted avg       0.77      0.76      0.75       937



# Other Machine Learning Classifiers.

In [ ]:
user = input("Enter a Text: ")
data = cv.transform([user]).toarray()
output = model.predict(data)
output1=LR.predict(data)
print("Bernoulli Naive Bayes:",output)
print("Logistic Regression:", output1)

Enter a Text: I am so tired
Bernoulli Naive Bayes: ['No Stress']
Logistic Regression: ['No Stress']


In [ ]:
user = input("Enter a Text: ")
data = cv.transform([user]).toarray()
output = model.predict(data)
output1=LR.predict(data)
print("Bernoulli Naive Bayes:",output)
print("Logistic Regression:", output1)

Enter a Text: I don't want to live.
Bernoulli Naive Bayes: ['No Stress']
Logistic Regression: ['No Stress']
